In [0]:
%run /Workspace/Users/surajss.3110@gmail.com/Learning/PySpark/MasterNoteBooks/Sales_MasterNoteBook

In [0]:
prod = load_product()
cust = load_customer()
trans = load_trans()

In [0]:
from pyspark.sql.functions import concat, lit, col

join = (
    trans
    .join(cust.alias("cust"), trans.customer_id == col("cust.customer_id"))
    .join(prod.alias("prod"), prod.product_id == trans.product_id)
    .withColumn("Customer_Name", concat(col("cust.first_name"), col("cust.last_name")))
    .select("transaction_id", col("cust.customer_id"), col("prod.product_id"), "Customer_Name")
)

display(join)


In [0]:
# 2. Create a New Column
# Add a column: total_amount = price × quantity

df = (trans.withColumn("total_amount" , round(col("price") * col("quantity"),2))
      .select("customer_id","product_id","price","quantity","total_amount"))

display(df)      


In [0]:
# 3. Find the top 5 customers based on total spending.

df = (trans
      .withColumn("total_amount" , round(col("price") * col("quantity"),2))
      .orderBy(desc("total_amount"))
      .select("customer_id","product_id","quantity","price","total_amount")
      .limit(5)
        )

display(df)       
      

In [0]:
# 4. Category Sales Summary For each category, calculate total revenue.

df = (
    trans
    .groupBy("category")
    .agg(round(sum(col("price") * col("quantity"))).alias("Total Revenue"))
    .orderBy(desc("Total Revenue"))
)

display(df)

In [0]:
#  Filter by Date

df = trans.filter("year(transaction_date)=2023")
display(df)



In [0]:
# City-Based Sales Report
# For each city, calculate:
# Number of transactions
# Total revenue
# Average order value

df = (trans
      .groupBy("city")
      .agg(
          count("*").alias("Number of transactions"),
          sum(col("price") * col("quantity")).alias("Total revenue"),
          avg(col("price") * col("quantity")).alias("Average order value")
      ))
      
display(df)
   

Databricks visualization. Run in Databricks to view.

In [0]:
# Find customers who purchased Electronics AND spent more than $2000 in total.

df = (trans
      .join(cust, cust.customer_id == trans.customer_id)
      .withColumn("Amount_Spend", (col("price") * col("quantity")))
      .select(trans.customer_id.alias("customer_id"), "transaction_date" ,"Amount_Spend")
      .filter("Amount_Spend > 2000")
      .filter("category == 'Electronics'")
      .orderBy(desc("Amount_Spend"))
      )

display(df)